In [ ]:
import gym
import theano
import theano.tensor as T
import numpy as np
import heapq
import ppaquette_gym_doom

In [ ]:
# Create a classic Doom environment with Gym
env = gym.make('ppaquette/DoomDefendCenter-v0')
env.reset()

In [ ]:
# Agent
class QAgent:
  def __init__(self, learn_rate=0.05, \
               action_set=set(),
               observation_encoder=lambda x: x, 
               verbose=False\):
    self.alpha = 0.9 # Probability of re-learning 
    self.policy = dict() # Q: (state,action) => value
    self.total_reward = 0
    self.learn_rate = learn_rate
    self.action_set = action_set
    self.observation_encoder = observation_encoder
    self.verbose = verbose

  """
  Predict the reward we would get
  if take [action] on [state]
  """
  def learn_Q(self, state, state_, action, reward):
    curr_Q  = Q(state, action)
    _,max_Q = find_best_action(state_)
    new_Q   = curr_Q + self.alpha*(reward + self.learn_rate * max_Q - curr_Q)
    if state in self.policy:
      self.policy[state][action] = reward # Always overwrite
    else: self.policy[state] = {action: new_Q}

  """
  Find maximum possible reward we would get
  from the best action attempted on [state]
  """
  def find_best_action(self, state):
    if state in self.policy:
      max_Q    = -1
      best_act = -1
      for a,r in self.policy[state].items():
        if r > maxQ:
          best_act = a
          max_Q = r
      return (best_act, max_Q)
    else: return (-1,-1)

  """
  Get recorded Q value of (state, action)
  """
  def Q(self, state, action):
    if state in self.policy:
      if action in self.policy[state]:
        return self.policy[state][action]
      else: return -1
    else: return -1
  
  @staticmethod
  def load(self,path):
    with open(path,'rb') as f:
      return pickle.load(f) 

  @staticmethod
  def save(self,path):
    with open(path,'rw') as f:
      return pickle.save(f, )



In [ ]:
# Play
bot = QAgent()
for _ in range(250):
  observation = env.render()
  # random_action = env.action_space.sample()
  observation, reward, done, info = env.step()
  if done:
    print('[Done]')
    print('...Total reward : {}'.format(bot.total_reward))
    break

In [ ]:
md5(3)